# Modelo Preditivo de Fraude - FINAL V-8.3

## Objetivo
Criar modelo preditivo usando como features os eventos associados à fraude reportada na planilha de POC.

## Observações
Cada observação corresponde à contagem diária de eventos ocorridos por agência, atm e tipo de evento.

## Entradas do Modelo (Features)
Neste modelo, são consideradas como features, dentre todos os eventos possíveis, os eventos selecionados de acordo com sua maior relevância na detecção de POC pelo modelo. Estes eventos foram obtidos a partir de um processo de feature selection executado na análise exploratória dos dados de log disponibilizados.
Tais eventos são lidos pelo aplicativo em produção a partir do arquivo descrito abaixo. desta forma, pode-se alterar as listas dos eventos alterando o conteúdo do arquivo, sem necessidade de alterar o aplictivo em produção. Recomenda-se que este arquivo seja alterado pelo responsável pelo modelo após testes e homologação, seguindo o procedimento definido para qualquer mudança de aplicativos em produção.
+ arquivo: poc_atm_features.csv
+ local: /sistema/bdf/ATM/
+ formato: uma única linha com os códigos numéricos de eventos válidos separados por vírgula, sem aspas.
+ período d eingestão: diário

O valor associado aos eventos, chamado de índice de proximidade da média, mede o percentual no qual o número de ocorrências do dado evento (feature) para uma dada agência/atm se afasta da média diária do dado evento para a mesma agência/atm.
+ fórmula:
    + iproxmed = (número de ocorrências do evento para agência+atm / média diária do evento para agência/atm).

## Entradas do Modelo (Target)
Identifica a ocorrência ou não de fraude para uma dada agência em uma determinada data, baseado em registro efetuado na data de identificação do evento na planilha BASE PCO ATM pela área de fraude.
Para obter este dado, o aplicativo lê dois arquivos. 
Um deles disponibilizado pela área com o registro das ocorrências de POCs em local definido para o processo de ingestão.
+ arquivo: poc-atm-registro_YYYY-MM-DD.csv
+ local: /sistema/bdf/ATM/
+ formato: 
    + Range Data: range de data onde a POC de determinada agência se encontra ativa 
    + Data Final do Comprometimento: data de final do comprometimento
    + Cod Terminal: código que identifica a ATM na agência
    + Data Inicial do Comprometimento
    + Agencia: código da agência
    + Nome Agência: nome da agência
    + Origem: Origem do registro do POC
    + Uso Fraudulento: descrição do uso fraudulento reportado
    + Golpe: descrição do golpe identificado
    + UF: UF da agência
+ periodidade de ingestão: diário

**IMPORTANTE:** Deve existir uma linha repetida para cada data do range no qual o POC permaneceu ativo. Por exemplo, se para um dado registro de POC a data início do POC foi 1/10/2016 e data de final do POC foi 3/10/2016, devem aparecer três linhas para a acorrência, apenas com a coluna data range diferente: 1/10/2016, 2/10/2016 e 3/10/2016.

**IMPORTANTE:** A coluna **origem** deve ser preenchida com o conteúdo **"Modelo de Preditor de Fraude"**, no caso em que a identificação da fraude se deu por meio das informações geradas pelo modelo preditor de fraude. Este feedback é muito importante para permitir o refinamento do modelo ao longo do tempo.


O outro sendo o registro diário de eventos em ATMs, sendo disponibilizado ao aplicativo por meio de tabela hive descrita abaixo:
+ tabela hive: poc_atm.events
+ formato: 
    + nr_sequ_even: código de identificação do evento
    + dh_even: data de ocorrência do evento
    + cd_eqpt: código de identificação da ATM na agência
    + nr_idef_secu: código da agência
+ periodidade de ingestão: diário

## Saídas do Modelo (Ranking de Agências Candidatas a POC e Marcadores de POC)
O modelo gera dois tipos de saída:
Um arquivo com o ranking das agências/ATMs candidatas a POC a partir da previsão do modelo.
+ arquivo: poc-atm-ranking_YYYY-MM-DD.csv 
+ local: /sistema/bdf/ATM/
+ formato: 
    + dh_even: data de ocorrência do evento
    + nr_idef_secu: código da agência
    + cd_eqpt: código de identificação da ATM na agência
    + nr_sequ_even: código de identificação do evento
    + probabilidade: probabilidade do evento ser relacionado a ocorrência de POC
+ ordem: decrescente de probabilidade

Um arquivo com os eventos marcadores de POC.
+ arquivo: poc-atm-marcador_YYYY-MM-DD.csv
+ local: /sistema/bdf/ATM/
    + formato: 
    + dh_even: data de ocorrência do evento
    + nr_idef_secu: código da agência
    + cd_eqpt: código de identificação da ATM na agência
    + nr_sequ_even: código de identificação do evento
    + nr_eventos: número de ocorrências do evento (acima do percentil de 95%)
    + percentil_max: valor do percentil 95% em relação ao número de ocorrências diárias do evento para uma dada agência/ATM
+ ordem: decrescente de data de referência, agência, ATM, evento           
+ periodidade de ingestão: diário

## Critério de Identificação de Eventos Marcadores de POC
Usando como baseline o arquivo de POCs reportados, serão identificados os eventos fora do padrão das agências/ATMs para serem usados como marcadores de POC no modelo preditivo.
São considerados fora do padrão, eventos que ocorrem em dias de ocorrência de POC, em agências reportadas como tendo ocorrência de POC, e que têm a quantidade de ocorrências no dia acima do percentil 95% da distribuição diária do eventos em todas as agências. Além disso, devem ser eventos que não aparecem com valores maiores que o percentil 95% nas agências em dias em que não existem ocorrências de POCs.

## Libraries

In [10]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
#from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

## Definição de constantes

In [11]:
# Load extrato from HDFS into Dataframe
# temporario para testas abaixo
#poc_in = '/user/x181472/export/V10_BASE_POC_ATM_060916.csv'
#ranking_poc_atm = '/temporario/poc_atm_ranking_'
#marcador_poc_atm = '/temporario/poc_atm_marcador_'
#features_poc_atm = '/temporario/poc_atm_features.csv'
#matriz_confusao = '/temporario/poc_atm_matriz_confusao_'
#auc = '/temporario/auc_poc_atm_'

poc = '/sistema/bdf/ATM/poc-atm-registro_'
ranking_poc_atm = '/sistema/bdf/ATM/poc-atm-ranking_'
marcador_poc_atm = '/sistema/bdf/ATM/poc-atm-marcador_'
features_poc_atm = '/sistema/bdf/ATM/poc_atm_features.csv'
matriz_confusao = '/sistema/bdf/ATM/poc_atm_matriz_confusao_'
auc = '/sistema/bdf/ATM/auc_poc_atm_'

mediana = 50  # mediana da ocorrência de eventos diários (feature)
percentil_max = 95 # percentil máximo considerado para treinar o modelo
D_90 = 90 # 90 dias a subtrair 
D_1 = 1 # obtem eventos de D-1 para limite superior de filtro por datas
D_5 = 6 # obtem eventos de D-6 para previsão de POC em produção

## Definição de funções

In [12]:
####################################################################################
# PARA RODAR EM HADOOP PRODUCAO DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO MAC OSX 
####################################################################################
# funcoes de tratamento de data
try:
    # o método replace permite o correto agrupamento por dia de uma data completa (com hora, minuto, segundo)
    funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                               .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
    funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

    # funcao de criacao de matriz de confusao
    def confusion_matrix(prevCol, labelCol):
        if prevCol == 1.0 and labelCol == 1.0:
            return 'TP'
        else:
            if prevCol == 1.0 and labelCol == 0.0:
                return 'FP'
            else:
                if prevCol == 0.0 and labelCol == 0.0:
                    return 'TN'
                else:
                    if prevCol == 0.0 and labelCol == 1.0:
                        return 'FN'

    # funcao que transforma lista em strings
    def listaString(l_even, dtype=str):
        return map(dtype, l_even)

except Exception as error:
        print('Erro -1: ' + repr(error))

## Carga de dados brutos
Carga dos dados brutos de log das ATMs e o de ocorrência de POCs.

### Trata Carga de Eventos de ATM

In [13]:
import datetime, time
from dateutil.relativedelta import relativedelta
try:
    # obtendo data atual como referencia
    data_ref = datetime.date.today()
    data_ref

    # obtendo faixa do período de histórico a partir da data de referencia 
    D_menos1 = data_ref + relativedelta(days=-D_1)
    D_menos5 = data_ref + relativedelta(days=-D_5)
    D_menos90 = D_menos1 + relativedelta(days=-D_90)
    #print "Data Ref :   " + str(data_ref)
    #print "D-90     :   " + str(D_menos90)
    #print "D-1      :   " + str(D_menos1)
    #print "D-5      :   " + str(D_menos5)

    #from datetime import datetime, time
    dt_sql_90 = (time.mktime(datetime.datetime.strptime(str(D_menos90), "%Y-%m-%d").timetuple()))
    dt_sql_1 = (time.mktime(datetime.datetime.strptime(str(D_menos1), "%Y-%m-%d").timetuple()))
    dt_sql_5 = (time.mktime(datetime.datetime.strptime(str(D_menos5), "%Y-%m-%d").timetuple()))
    dt_sql_ref = (time.mktime(datetime.datetime.strptime(str(data_ref), "%Y-%m-%d").timetuple()))
    #print dt_sql_90
    #print dt_sql_1
    #print dt_sql_5
    #print dt_sql_ref
except Exception as error:
        print('Erro -1: ' + repr(error))

In [14]:
try:
    rdd_eventlog = hive_context.table('poc_atm.events')

    if rdd_eventlog.count() == 0:
        raise Exception("erro na leitura da tabela poc_atm.events")
        
except Exception as error:
        print('Erro -2: ' + repr(error))

try:
    df_eventlog = rdd_eventlog \
        .map(lambda p: (p[1],p[2],p[3], p[4])) \
        .toDF(['nr_sequ_even',
               'dh_even', 
               'cd_eqpt', 
               'nr_idef_secu' 
                   ])

    #print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

    # sumarizar a base por agencia+atm+evento+dia
    #df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

    df_eventlog = df_eventlog\
        .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
        .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
        .withColumnRenamed('count', 'nr_eventos')

    df_eventlog = df_eventlog \
        .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
        .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
        .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
        .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
        .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

    # Elimina NAs
    df_eventlog = df_eventlog.dropna()

    # FILTRANDO SOMENTE ULTIMOS 90 DIAS, incluindo log de D-2
    df_eventlog2 = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                   .format(dt_sql_90, dt_sql_1)).orderBy("dh_even")
except Exception as error:
        print('Erro -3: ' + repr(error))

KeyboardInterrupt: 

In [ ]:
df_eventlog2.count()

## Calculando Percentil de Eventos Marcadores de POC

In [8]:
# calculando percentis de baseline para todas as observações do log com base em agencia/evento
# OBS: o uso do percentil desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem
try: 
    df_eventlog_percen = df_eventlog2.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

    rdd_eventlog_percen = df_eventlog_percen.rdd \
        .map(lambda x: (x[0], (x[1],), )) \
        .reduceByKey(lambda acc, val: acc + val) \
        .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

    rdd_eventlog_percen = rdd_eventlog_percen \
       .map(lambda k: (k[0].split("-"), k[1], k[2])) 

    # dataframe por agencia/evento com valor de percentil_max
    df_eventlog_percen = rdd_eventlog_percen \
        .map(lambda p: (p[0][0],p[0][1], float(p[2])  )) \
        .toDF(['nr_idef_secu',
               'nr_sequ_even',
               'percentil_max'
                   ])

    # colocar no log total as colunas de mediana e percentil máximo por agencia+evento 
    df_eventlog2 = df_eventlog2 \
        .join(df_eventlog_percen, (df_eventlog2['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_eventlog2['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
        .drop(df_eventlog_percen["nr_idef_secu"])\
        .drop(df_eventlog_percen["nr_sequ_even"])

    df_eventlog2 = df_eventlog2.dropna()

    # tirando zeros a esquerda e truncando codigos de agencia e atm
    df_eventlog2 = df_eventlog2\
        .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
        .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda

except Exception as error:
    print('Erro -4: ' + repr(error))

KeyboardInterrupt: 

In [9]:
df_eventlog2.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



### Dados de registro de POC
Trata as seguintes colunas:
+ range de data de POC ativo
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC
+ descrição do tipo de golpe aplicado

In [6]:
try:
    poc_in = poc + data_ref.strftime('%Y-%m-%d') + ".csv"
    rdd_poc = sc.textFile(poc_in)

    if rdd_poc.count() == 0:
        raise Exception("erro na leitura da tabela V10_BASE_POC_ATM_060916.csv")
        
except Exception as error:
        print('Erro -5: ' + repr(error))

try:
    header = rdd_poc.first()
    # pegando data inicial de comprometimento ao inves de data_cad, pois analise exploratoria mostrou que tem
    # mais sentido usar esta data
    df_poc = rdd_poc \
        .filter(lambda line: line != header) \
        .map(lambda k: k.split(";")) \
        .map(lambda p: (p[0], p[3], p[7])) \
        .toDF(['data_range', 
                'agencia',
                'golpe'
        ])

    df_poc = df_poc.withColumn('data_range', funcStrToDate_dmyyyy(col('data_range'))) 

    df_poc = df_poc \
        .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
        .withColumn('data_range', df_poc["data_range"].cast(TimestampType())) \
        .withColumn('golpe', df_poc["golpe"].cast(StringType()))

    # FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
    df_poc = df_poc.where("CAST(data_range AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_90,dt_sql_1)).orderBy("data_range")

    # removing NAs
    df_poc = df_poc.dropna()
    
except Exception as error:
        print('Erro -6: ' + repr(error))



Erro -5: NameError("name 'poc_in' is not defined",)
Erro -6: NameError("name 'rdd_poc' is not defined",)


### Selecionando Eventos por ATM/Agência em dias POC

In [7]:
# SELECIONANDO DO LOG TOTAL SOMENTE AGENCIAS E DIAS DE OCORRENCIA DE POC ACIMA DO PERCENTIL
try:
    df_poceven = df_eventlog2 \
        .join(df_poc, (df_poc['data_range'] == df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'left')

    # LOG SOMENTE COM AGENCIAS POC NAS DATAS POC E MAIOR QUE PERCENTIL
    df_pocevenSIM = df_poceven\
        .filter((col("nr_eventos") > col('percentil_max')) &
            ((col("agencia") == col('nr_idef_secu'))))
        
except Exception as error:
    print('Erro -7: ' + repr(error))

Erro -7: NameError("name 'df_eventlog2' is not defined",)


### Selecionando Eventos por ATM/Agência Fora de dias POC

In [8]:
# LOG SOMENTE COM TODAS AS AGENCIAS NAS DATAS FORA DE POC E MENOR QUE PERCENTIL
# join left garante pegar eventos que ocorrem em dia de POC e não depois
try:
    df_pocevenNAO = df_poceven\
        .filter((col("nr_eventos") <= col('percentil_max')) &
                (col('agencia').isNull())) # indica que foi fora de dia de POC

    # JOIN DAS DUAS ACIMA POR EVENTO PARA OBTER LISTA DE POKEVENTOS
    df_pokeventos = df_pocevenSIM\
        .join(df_pocevenNAO,
              (df_pocevenSIM['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'left')\
        .drop(df_pocevenNAO["nr_sequ_even"])\
        .groupBy('nr_sequ_even').count()
        
    if df_pokeventos.count() == 0:
        raise Exception("Nao existe nenhum evento marcador de POC no periodo")
        
except Exception as error:
    print('Erro -8: ' + repr(error))

Erro -8: NameError("name 'df_poceven' is not defined",)


### Obtendo Lista de Features
Carrega os eventos a serem usados como features, a partir do arquivo de configuração 'features_poc_atm.csv, que contém a lista de eventos a serem usadas no modelo

In [9]:
# lendo arquivo de pokeventos e carregando em lista
try:
    pd_evenfeat = pd.read_csv(features_poc_atm, header=None).T

    if pd_evenfeat.count()[0] == 0:
        raise Exception("erro na leitura da tabela poc_atm_features.csv")
        

    pd_evenfeat.columns = ['features']
    #pd_evenfeat
    l_feat = pd_evenfeat.features.tolist()

    l_features = listaString(l_feat)
    #l_features
    # abaixo comentado pra usar arquivo configurável
    #l_features1 = ["5840", "5251", "1649", "1746", "5242", "6740", "2184", "5241", "5248", "5198"] 
except Exception as error:
        print('Erro -9: ' + repr(error))

Erro -9: NameError("name 'pd' is not defined",)


### Prepara Dados de Uso para gerar Ranking

In [14]:
# USO 1
# FILTRANDO SOMENTE de D-1 a D-6
# TAMBEM SELECIONAR AQUI SOMENTE AGENCIAS QUE TEM OCORRENCIA DOS POKEVENTOS (AO MENOS 1)
try:
    df_eventlog_USO = df_eventlog2.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_5,dt_sql_1)).orderBy("dh_even")
    df_eventlog_USO = df_eventlog_USO.filter((df_eventlog_USO["nr_sequ_even"].isin(l_features)))

    if df_eventlog_USO.count() == 0:
        raise Exception("Nao existe nenhum evento relevante entre D-1 e D-6")
        
except Exception as error:
    print('Erro -10: ' + repr(error))

Erro -10: NameError("name 'df_eventlog2' is not defined",)


### Obtém e Salva arquivo de Marcadores de POC

In [23]:
# Seleciona de D-1 somente linhas com pokeventos
try:
    l_marcapoc = df_pokeventos.toPandas().nr_sequ_even.tolist()
    df_marcaPOC = df_eventlog_USO\
        .filter((df_eventlog_USO["nr_sequ_even"].isin(l_marcapoc)))
    #df_marcaPOC.toPandas()

    # filtra somente linhas com nr_eventos > percentil_max para agência/atm/evento na faixa de data considerada
    # OBS: em produição está faixa estará restrita a D-1
    df_marcaPOC = df_marcaPOC\
        .filter(col("nr_eventos") > col('percentil_max'))\
        .orderBy(col('dh_even').desc(), col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even')) 
        # indica que foi fora de dia de POC

    #df_marcaPOC.toPandas()

    # salvando em arquivo os marcadores de poc
    #marcador_poc_atm_out = marcador_poc_atm + str(data_ref) + ".csv"
    marcador_poc_atm_out = marcador_poc_atm + data_ref.strftime('%Y-%m-%d') + ".csv"
    df_marcaPOC.toPandas().to_csv(marcador_poc_atm_out)

except Exception as error:
    print('Erro -10: ' + repr(error))

Erro -10: NameError("name 'df_pokeventos' is not defined",)


## Preparando Dados Processados para Treino do Modelo
Preparação do dataframe com as features e o target para treino do modelo


In [9]:
try:
    df_eventlog_TRN = df_eventlog2.filter((df_eventlog2["nr_sequ_even"].isin(l_features)))

    # obter as colunas de med diária por ag/equip/evento a partir do log todo
    # media
    df_even_med = df_eventlog2\
        .groupBy(['nr_idef_secu','cd_eqpt','nr_sequ_even'])\
        .avg('nr_eventos').withColumnRenamed('avg(nr_eventos)', 'med')

    # concatenar dados de features calculadas com dados reais agrupados por ag/equip/evento
    df_feat_TRN = df_eventlog_TRN \
        .join(df_even_med, (df_eventlog_TRN['nr_idef_secu'] == df_even_med['nr_idef_secu']) &
                          (df_eventlog_TRN['cd_eqpt'] == df_even_med['cd_eqpt']) &
                          (df_eventlog_TRN['nr_sequ_even'] == df_even_med['nr_sequ_even']), 'left') \
        .drop(df_even_med["nr_idef_secu"]).drop(df_even_med["cd_eqpt"]).drop(df_even_med["nr_sequ_even"])

    # cria coluna de indice de proximidade da media (iproxmed)
    # calculado como número de ocorrências real diaria/média diaria, por agencia e atm
    df_feat_TRN = df_feat_TRN\
        .withColumn('iproxmed', col("nr_eventos")/col("med"))

    # transpondo tipo de evento para gerar features
    df_feat_TRN = df_feat_TRN\
        .drop(df_feat_TRN["med"]).drop(df_feat_TRN["nr_eventos"])

    df_feat_TRN = df_feat_TRN.dropna()

    pd_tmp_TRN = df_feat_TRN.toPandas()

    # transposição do dataframe
    pd_feat_TRN = pd.pivot_table(pd_tmp_TRN,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

    pd_feat_TRN.columns = [c[1] if c[1] else c[0] for c in pd_feat_TRN.columns.tolist()]
    pd_feat_TRN = pd_feat_TRN.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') )\
        .drop('dh_even', axis=1)
    #pd_feat_TRN

    # lista de features para treinar o modelo
    # a partir das features restantes apos a transposicao
    l_feat_TRN = pd_feat_TRN.columns.tolist()
    l_feat_final = l_feat_TRN[2:-1]

    # transforma de Pandas para dataframe
    df_feat_alt_TRN = sqlCtx.createDataFrame(pd_feat_TRN)

    # reordenando as colunas
    df_feat_alt_TRN = df_feat_alt_TRN.select('nr_idef_secu', 'cd_eqpt','data_even', 
                                           *[col for col in df_feat_alt_TRN.columns if (col != 'nr_idef_secu') &
                                     (col != 'cd_eqpt') &
                                     (col != 'data_even')])

    # removendo o dataframe Pandas da memoria
    lst = [pd_feat_TRN]
    del pd_feat_TRN # dfs still in list
    del lst     # memory release now

    df_feat_alt_TRN = df_feat_alt_TRN.withColumn('data_even', df_feat_alt_TRN["data_even"].cast(TimestampType()))

    df_tidy_TRN = df_feat_alt_TRN\
        .withColumn('nr_idef_secu', trim(df_feat_alt_TRN.nr_idef_secu))\
        .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zeros  a esquerda

    ################################# 
    df_tidy_TRN = df_tidy_TRN.dropna()

    df_poc = df_poc\
        .withColumn('agencia', trim(df_poc.agencia))\
        .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zeros  a esquerda

    df_poc.printSchema()

    df_tidy_TRN.count()

    # se achar, marca como POC = '1.0', concatenando por data e agencia
    df_tidy_TRN = df_tidy_TRN \
        .join(df_poc, (df_tidy_TRN['nr_idef_secu'] == df_poc['agencia']) &
              (df_tidy_TRN['data_even'] == df_poc['data_range']), 'left')

    df_tidy_TRN = df_tidy_TRN.withColumn("label",
            when(df_tidy_TRN["nr_idef_secu"] == df_poc["agencia"],1.0) \
        .otherwise(0.0))\
        .drop(df_poc["agencia"]).drop(df_poc["data_range"]).drop(df_poc["golpe"])

    # força tipo double nas colunas que a trsnsposição criou como long (motivo desconhecido ainda)
    l_primeven = l_feat_final[0]
    df_tidy_dbl_TRN = df_tidy_TRN\
       .withColumn(l_primeven, df_tidy_TRN[l_primeven].cast(DoubleType()))\

    for even in l_feat_final[1:]:
        df_tidy_dbl_TRN = df_tidy_dbl_TRN.withColumn(even, df_tidy_dbl_TRN[even].cast(DoubleType()))

except Exception as error:
    print('Erro -11: ' + repr(error))

Erro -11: NameError("name 'df_eventlog2' is not defined",)


## Modelagem com Randon Forest (library ML)

### Preparação do dados para treino do modelo

In [10]:
try:
    df_tidy_dbl_TRN = df_tidy_dbl_TRN.withColumn('data_even', df_tidy_dbl_TRN["data_even"].cast(TimestampType()))

#    from pyspark.mllib.linalg import Vectors
#    from pyspark.ml.feature import VectorAssembler

#    from pyspark.ml import Pipeline
#    from pyspark.ml.classification import RandomForestClassifier
#    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#    from pyspark.ml.evaluation import MulticlassClassificationEvaluator

    # rodando para as features ocm mais relevância já analisadas
    assemblerTrain = VectorAssembler(
        inputCols=l_feat_final,
        outputCol="features")

    df_tmp = assemblerTrain.transform(df_tidy_dbl_TRN)

    df_tidy_rf = df_tmp.select("data_even","nr_idef_secu", "cd_eqpt", "label", "features")

    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_rf)

    # Automatically identify categorical features, and index them.
    # We specify maxCategories so features with > 2 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_rf)

    # Split the data into training and test sets (30% held out for testing)
    (trainData_LPB, testData_LPB) = df_tidy_rf.randomSplit([0.7, 0.3])

except Exception as error:
    print('Erro -12: ' + repr(error))

Erro -12: NameError("name 'df_tidy_dbl_TRN' is not defined",)


### Treino e Validação do Modelo

In [17]:
try:
    # Train a Random Forest model.
    rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

    # DESCOMENTAR ABAIXO PARA USO DE CROSS VALIDATION
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
    #    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    #    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

    numFolds = 2

    evaluator = BinaryClassificationEvaluator()

    crossval = CrossValidator(
        estimator=pipeline,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        numFolds=numFolds)

    model_ml = crossval.fit(trainData_LPB)

    # Fetch best model
    #tree_model = model.bestModel.stages[2]
    #print tree_model

    #from pyspark.ml.evaluation import BinaryClassificationEvaluator
    predictions = model_ml.transform(testData_LPB)
    #predictions.take(2)
    #evaluator = BinaryClassificationEvaluator()

    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    evaluator.explainParams()
    
    auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

    accuracy = evaluator.evaluate(predictions)

    #Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

    createNewColFromTwo = udf(confusion_matrix, StringType())
    df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'],
            predictions['label']))

    df_rates = df_conf_matr.groupBy("conf_matr").count()
    df_rates.toPandas()

except Exception as error:
    print('Erro -13: ' + repr(error))

Erro -13: NameError("name 'RandomForestClassifier' is not defined",)


### Salva Logs de Performance do Modelo
Salva os arquivos de log abaixo com dados da performance do treino do modelo a cada execução:
+ auc_poc_atm<data de referência> : área sob a curva (AUC) e acurácia obtida no treino do modelo na data de referência
+ matriz_confusao-<data de referência> : matriz de confusão obtida no treino do modelo na data de referência

In [18]:
try:
    # gera arquivo de log com matriz de confusao para cada data de referencia
    df_rates = df_rates\
       .withColumn('data_ref', lit(data_ref))

    matriz_confusao_out = matriz_confusao + data_ref.strftime('%Y-%m-%d')
    df_rates.toPandas().to_csv(matriz_confusao_out)

    # gera arquivo de log com AUC para cada data de referencia
    rdd_auc = sc.parallelize([(data_ref, auroc, accuracy)])
    schema = StructType([StructField("data_ref", TimestampType(), True),
                         StructField("AUC", DoubleType(), True),
                         StructField("acuracia", DoubleType(), True)
                    ])
    df_auc = sqlContext.createDataFrame(rdd_auc, schema)

    auc_out = auc + + data_ref.strftime('%Y-%m-%d')
    df_auc.toPandas().to_csv(auc_out)
except Exception as error:
    print('Erro -14: ' + repr(error))

Erro -14: NameError("name 'df_rates' is not defined",)


## Uso do modelo

### Preparação dos Dados

In [19]:
try:
    # concatenar dados de features calculadas com dados reais agrupados por ag/equip/evento
    df_feat_USO = df_eventlog_USO \
        .join(df_even_med, (df_eventlog_USO['nr_idef_secu'] == df_even_med['nr_idef_secu']) &
                          (df_eventlog_USO['cd_eqpt'] == df_even_med['cd_eqpt']) &
                          (df_eventlog_USO['nr_sequ_even'] == df_even_med['nr_sequ_even']), 'left')\
        .drop(df_even_med["nr_idef_secu"]).drop(df_even_med["cd_eqpt"]).drop(df_even_med["nr_sequ_even"])

    df_feat_USO = df_feat_USO.dropna()

    # criar coluna de indice de proximidade da media (iproxmed)
    # calculado como número de ocorrências real diaria/média diaria, por agencia e atm
    df_feat_USO = df_feat_USO\
       .withColumn('iproxmed', df_feat_USO["nr_eventos"]/df_feat_USO["med"])

    # transpondo tipo de evento para gerar features
    df_feat_USO = df_feat_USO\
        .drop(df_feat_USO["med"]).drop(df_feat_USO["nr_eventos"])

    pd_tmp_USO = df_feat_USO.select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

    pd_feat_USO = pd.pivot_table(pd_tmp_USO,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

    pd_feat_USO.columns = [c[1] if c[1] else c[0] for c in pd_feat_USO.columns.tolist()]
    pd_feat_USO = pd_feat_USO.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') )\
        .drop('dh_even', axis=1)

    # cria dataframe a partir de Pandas
    df_feat_alt_USO = sqlCtx.createDataFrame(pd_feat_USO)

    # reordenando as colunas
    df_feat_alt_USO = df_feat_alt_USO.select('nr_idef_secu', 'cd_eqpt','data_even', 
                                           *[col for col in df_feat_alt_USO.columns if (col != 'nr_idef_secu') &
                                     (col != 'cd_eqpt') &
                                     (col != 'data_even')])

    df_feat_alt_USO = df_feat_alt_USO.withColumn('data_even', df_feat_alt_USO["data_even"].cast(TimestampType()))

    df_tidy_USO = df_feat_alt_USO\
        .withColumn('nr_idef_secu', trim(df_feat_alt_USO.nr_idef_secu))\
        .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zeros  a esquerda

    # insere colunas que possam faltar nos dados de uso
    for even in list(set(df_tidy_dbl_TRN.columns[3:-1]).difference(df_tidy_USO.columns[3:])):
        df_tidy_USO = df_tidy_USO.withColumn(even, lit(0))
        df_tidy_USO = df_tidy_USO.withColumn(even, df_tidy_USO[even].cast(DoubleType()))

    # força label = 0.0 (sem uso, somente para aplicar funcao predict para uso)
    df_tidy_USO = df_tidy_USO.withColumn("label", lit(0))
    df_tidy_USO = df_tidy_USO.withColumn('label', df_tidy_USO["label"].cast(DoubleType()))

    # força tipo double nas colunas que a trsnsposição criou como long (motivo desconhecido ainda)
    l_primeven = l_feat_final[0]
    df_tidy_dbl_USO = df_tidy_USO\
       .withColumn(l_primeven, df_tidy_USO[l_primeven].cast(DoubleType()))\

    for even in l_feat_final[1:]:
        df_tidy_dbl_USO = df_tidy_dbl_USO.withColumn(even, df_tidy_dbl_USO[even].cast(DoubleType()))

except Exception as error:
    print('Erro -15: ' + repr(error))

Erro -15: NameError("name 'df_eventlog_USO' is not defined",)


### Aplicação do Modelo

In [20]:
try:
    df_tmp = assemblerTrain.transform(df_tidy_dbl_USO)

    df_tidy_uso = df_tmp.select("nr_idef_secu", "cd_eqpt", "data_even", "label", "features")

    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_uso)

    # Automatically identify categorical features, and index them.
    # We specify maxCategories so features with > 2 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_uso)

    # Split the data into training and test sets (30% held out for testing)
    predictions = model_ml.transform(df_tidy_uso)

    predictions.map(lambda p: (p[2],p[0],p[1],p[3], p[8][1] )).first()

    df_rankingFinal = predictions.map(lambda p: (p[2],p[0],p[1],p[3], str(p[8][1]) ))\
        .toDF(['dh_even',
               'nr_idef_secu', 
               'cd_eqpt', 
               'label',
               'probabilidade'
                   ])

    #from pyspark.sql.functions import col,udf, unix_timestamp
    df_rankingFinal = df_rankingFinal\
        .withColumn('probabilidade', df_rankingFinal["probabilidade"].cast(DoubleType()))\
        .orderBy(col('probabilidade').desc())\
        .drop(df_rankingFinal["label"])

    # salvando ranking
    ranking_poc_atm_out = ranking_poc_atm + data_ref.strftime('%Y-%m-%d') + ".csv"
    df_rankingFinal.toPandas().to_csv(ranking_poc_atm_out)

except Exception as error:
    print('Erro -16: ' + repr(error))

Erro -16: NameError("name 'assemblerTrain' is not defined",)
